# **FUNCIONES AUXILIARES**

---
Este notebook contiene funciones auxiliares que se utilizan durante el
entrenamiento de la red neuronal convolucional


## **1. Importación de librerías**

In [ ]:
import copy
import math
import numpy as np
import itertools
import matplotlib.pyplot as plt
from tensorflow.compat.v1.keras.preprocessing.image import ImageDataGenerator

## **2. Funciones para cronograma tasa de aprendizaje**

---
Se crean funciones para variar la tase de aprendizaje en cada época del entrenamiento de la red neuronal convolucional, al cual se puede acceder a través del siguiente link: [Learning Rate Scheduler with MXNet library ](https://mxnet.apache.org/versions/1.7/api/python/docs/tutorials/packages/gluon/training/learning_rates/learning_rate_schedules_advanced.html)


### **2.1 Variación triangular de tasa de aprendizaje**

In [ ]:
class TriangularSchedule():
    def __init__(self, min_lr, max_lr, cycle_length, inc_fraction=0.5):     
        """
        Esta clase permite configurar una variación de la tasa de aprendizaje
        con una cronograma triangular.

        Parámetros de entrada:
        min_lr: límite inferior para la tasa de aprendizaje (float)
        max_lr: límite superior para la tasa de aprendizaje (float)
        cycle_length: número de iteraciones entre el inicio y el final (int)
        inc_fraction: fracción de iteraciones utilizadas en la etapa (float)

        Parámetros de salida:
        adjusted_cycle: valor de la tasa de aprendizaje




        """
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.cycle_length = cycle_length
        self.inc_fraction = inc_fraction
        
    def __call__(self, iteration):
        if iteration <= self.cycle_length*self.inc_fraction:
            unit_cycle = iteration * 1 / (self.cycle_length * self.inc_fraction)
        elif iteration <= self.cycle_length:
            unit_cycle = (self.cycle_length - iteration) * 1 / (self.cycle_length * (1 - self.inc_fraction))
        else:
            unit_cycle = 0
        adjusted_cycle = (unit_cycle * (self.max_lr - self.min_lr)) + self.min_lr
        return adjusted_cycle

### **2.2 Variación Cosine Annealing de  la tasa de aprendizaje**

In [ ]:
class CosineAnnealingSchedule():
    def __init__(self, min_lr, max_lr, cycle_length):
        """
        Esta clase permite configurar una variación de la tasa de aprendizaje
        con una cronograma CosineAnnealing.


        Parámetro de entrada:
        min_lr: límite inferior para la tasa de aprendizaje (float)
        max_lr: límite superior para la tasa de aprendizaje(float)
        cycle_length: número de iteraciones entre el inicio y el final (int)

        Parámetros de salida:
        min_lr: valor de la tasa de aprendizaje



        """
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.cycle_length = cycle_length
        
    def __call__(self, iteration):
        if iteration <= self.cycle_length:
            unit_cycle = (1 + math.cos(iteration * math.pi / self.cycle_length)) / 2
            adjusted_cycle = (unit_cycle * (self.max_lr - self.min_lr)) + self.min_lr
            return adjusted_cycle
        else:
            return self.min_lr

### **2.3 Creación de cronograma de tasa de aprendizaje cíclico**

In [ ]:
class CyclicalSchedule():

    def __init__(self, schedule_class, cycle_length, cycle_length_decay=1, cycle_magnitude_decay=1, **kwargs):
        """

        Esta clase permite realizar una variación de la tasa de aprendizaje de
        forma cíclica a partir de un cronograma específico.

        Parámetros de entrada:
        schedule_class: Cronograma de la tasa de aprendizaje (Clase)
        cycle_length: iteraciones utilizadas para el ciclo inicial (int)
        cycle_length_decay: factor multiplicado por cycle_length en cada ciclo (float)
        cycle_magnitude_decay: factor multiplicado la tasa de aprendizaje magnitudes en cada ciclo (float)
        kwargs: parámetros adicionales que necesita schedule_class

        Parámetros de salida:
        valor de la tasa de aprendizaje

        """
        self.schedule_class = schedule_class
        self.length = cycle_length
        self.length_decay = cycle_length_decay
        self.magnitude_decay = cycle_magnitude_decay
        self.kwargs = kwargs
    
    def __call__(self, iteration):
        cycle_idx = 0
        cycle_length = self.length
        idx = self.length
        while idx <= iteration:
            cycle_length = math.ceil(cycle_length * self.length_decay)
            cycle_idx += 1
            idx += cycle_length
        cycle_offset = iteration - idx + cycle_length
        
        schedule = self.schedule_class(cycle_length=cycle_length, **self.kwargs)
        return schedule(cycle_offset) * self.magnitude_decay**cycle_idx

## **3. Función para gráficar una matriz de confusión**

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    Esta función imprime y traza la matriz de confusión.
    Se puede normalizar la matriz de confunsión aplicando `normalize = True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.6f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

## **4. Iteración de imágenes**


---
Esta clase itera un lote de imágenes  para evitar cargar  en memoria todo el conjunto completo de imágenes. 
La iteración retorna  dos lotes  de imágenes del mismo tamaño  simultáneamente, para ser aprovechada por la entrada de la red neuronal convolucional.


In [ ]:
class MultipleGenerator():

  def __init__ (self,dataframe,directory,x_col1,x_col2,y_col,target_size, class_mode,batch_size):
    """

        Esta clase permite construir un iterador de imágenes por lote de 
        dos entradas.

        Parámetros de entrada:
        dataframe: Panda dataframe con información del nombre de las imágenes
        directory: ruta principal donde se encuentran las imágenes
        x_col1: columna en el Panda dataframe con los nombres de las imágenes para el primer lote
        x_col2: columna en el Panda dataframe con los nombres de las imágenes para el segundo lote
        y_col: columna en el Panda dataframe con las etiquetas de las imágenes
        target_size: dimensiones de la imagen (tuple)
        batch_size: tamaño de cada lote de imágenes

        Parámetros de salida:
        Lote de imágenes en formato Numpy Array
        """
    print('Vertical Spectre Images')
    self.Branch1_batches = ImageDataGenerator().flow_from_dataframe(
        dataframe = dataframe,            
        directory = directory,
        x_col = x_col1,
        y_col = y_col,
        target_size = target_size,
        class_mode = class_mode,
        batch_size = batch_size,
        shuffle = False)
    
    print('Horizontal Spectre Images')
    self.Branch2_batches = ImageDataGenerator().flow_from_dataframe(
        dataframe = dataframe,            
        directory = directory,
        x_col = x_col2,
        y_col = y_col,
        target_size = target_size,
        class_mode = class_mode,
        batch_size = batch_size,
        shuffle = False)
    
  def two_inputs(self):
    while True:
      X1i = self.Branch1_batches.next()
      X2i = self.Branch2_batches.next()
      yield [X1i[0], X2i[0]], X2i[1]

  def labels_classes(self):
    return self.Branch2_batches.classes

  def genreset(self):
    self.Branch1_batches.reset()
    self.Branch2_batches.reset()
